In [61]:
import pandas as pd 
import string
import numpy as np

df = pd.read_csv('fp2.csv')


In [62]:


def clean(col1, col2):
    df[col1] = df[col2].astype(str) + df[col1].astype(str)
    df[col1] = [w.replace('nan', '').strip('[]\'').replace("'", '').replace(', ', ';') for w in df[col1]]

for c1, c2 in zip(['Title', 'Year', 'Publisher', 'Genre'], ['Title:', 'Release Date:', 'Publisher:', 'Genre:']):
    clean(c1, c2)
    
df['Year'] = [y[-4:] if len(y) > 4 else y for y in df['Year']]
df['No_Players'] = [str(w).strip(string.ascii_letters+' ') if isinstance(w, str) else w for w in df['No_Players']]
df['No_Players'] = [entry.split('-')[1] if isinstance(entry, str) and '-' in entry else entry for entry in df['No_Players']]        
df = df[df['Percentage of Positive Reviews'].notna()]
df['Franchise:'] = [w.strip('[]\'') if isinstance(w, str) else w for w in df['Franchise:']]
df['Developer:'] = [w.strip('[]\'') if isinstance(w, str) else w for w in df['Developer:']]
df['Genre'] = [w.replace(']', ';') for w in df['Genre']]
df = df.drop(['Title:', 'Release Date:', 'Publisher:', 'Genre:', 'Publisher', 'Developer:', 'Discounted Price', 'Unnamed: 0'], axis=1)
df = df.dropna(thresh=6)

def x(s):
    if s != 0:
        return 1
    else:
        return 0

df['Franchise:'] = df['Franchise:'].fillna(0)
df['Franchise:'] = df['Franchise:'].map(x)
df['Percentage of Positive Reviews'] = [w.strip('%') if isinstance(w, str) else w for w in df['Percentage of Positive Reviews']]
df['Price'] = [w.strip('$') if isinstance(w, str) else w for w in df['Price']]
df = df[df['Percentage of Positive Reviews'] != 'No Rating']
# remove percentage and dollar sign, add genre columns

In [63]:
import re
def find_average(series):
    total = 0
    length = 0
    for e in series:
        try:
            val = str(e)
            loi = [s for s in val if s.isdigit()]
            if loi:
                total += int(''.join(loi))
                length += 1
        except ValueError:
            try:
                total += val
                length += 1
            except ValueError:
                print('error')
                print(type(e))
                print(e)
                
    return total/length
meta_avg = find_average(df['Metascore'])
user_avg = find_average(df['Avg_Userscore'])
player_avg = find_average(df['No_Players'])

df['Metascore'].fillna(round(meta_avg), inplace=True)
df['Avg_Userscore'].fillna(user_avg/10., inplace=True)
df['No_Players'].fillna(round(player_avg), inplace=True)
df['Platform'].fillna('', inplace=True)
df

,Title,Year,Genre,Platform,Metascore,Avg_Userscore,No_Players,Number of Ratings,Percentage of Positive Reviews,Price,Franchise:
13,GrandTheftAutoV]Grand Theft Auto V,2014,Action;Adventure;Action Adventure;Modern;Open-...,XboxOne,97,7.8,30,773837.0,79,29.99,1
14,GrandTheftAutoV]Grand Theft Auto V,2013,Action;Adventure;Modern;Action Adventure;Moder...,PlayStation3,97,8.3,16,773837.0,79,29.99,1
15,GrandTheftAutoV]Grand Theft Auto V,2013,Action;Adventure;Modern;Action Adventure;Moder...,Xbox360,97,8.3,16,773837.0,79,29.99,1
16,GrandTheftAutoV]Grand Theft Auto V,2014,Action;Adventure;Action Adventure;Modern;Open-...,PlayStation4,97,8.3,30,773837.0,79,29.99,1
17,GrandTheftAutoV]Grand Theft Auto V,2015,Action;Adventure;Modern;Action Adventure;Open-...,PC,96,7.7,32,773837.0,79,29.99,1
...,...,...,...,...,...,...,...,...,...,...,...
5846,ZooTycoonUltimateAnimalCollection,2018,Simulation,,85,7.7,8,598.0,57,19.99,0
5847,KingdomUnderFire2,2019,Action;Massively Multiplayer;RPG;Strategy,,85,7.7,8,1399.0,73,9.99,0
5848,Thief,2014,Action;Adventure,,85,7.7,8,11296.0,72,19.99,1
5849,STARWARSTheForceUnleashedUltimateSithEdition,2009,Action,,85,7.7,8,3805.0,69,19.99,1


In [64]:
s = df['Genre']
s = [entry.split(';') for  entry in s.values]

In [65]:
def count_unique(series):
    
    unique_items = []
    
    for item in series:
        for i in range(len(item)):
            
            if item[i] not in unique_items:
                
                unique_items.append(item[i])
    unique_items = {item:i for i, item in enumerate(unique_items)}
    return unique_items


In [66]:
import numpy as np
def one_hot(series, unique_lst):
    encoded = np.zeros((len(series), len(unique_lst)))
    for i, genres in enumerate(series):
        encoded[i][[unique_lst[g] for g in genres]] = 1
        
    return encoded



In [68]:
unique = count_unique(s)
genres = one_hot(s, unique)
for g in unique.keys():
    if sum(genres[:, unique[g]]) >= 5:
        df[g] = genres[:, unique[g]]
df = df.drop('Genre', axis=1)

df

,Title,Year,Platform,Metascore,Avg_Userscore,No_Players,Number of Ratings,Percentage of Positive Reviews,Price,Franchise:,...,Fighting,Fantasy,Racing,Sports,Massively Multiplayer,Casual,Sandbox,Survival,Early Access,Design & Illustration
13,GrandTheftAutoV]Grand Theft Auto V,2014,XboxOne,97,7.8,30,773837.0,79,29.99,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,GrandTheftAutoV]Grand Theft Auto V,2013,PlayStation3,97,8.3,16,773837.0,79,29.99,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,GrandTheftAutoV]Grand Theft Auto V,2013,Xbox360,97,8.3,16,773837.0,79,29.99,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,GrandTheftAutoV]Grand Theft Auto V,2014,PlayStation4,97,8.3,30,773837.0,79,29.99,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,GrandTheftAutoV]Grand Theft Auto V,2015,PC,96,7.7,32,773837.0,79,29.99,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5846,ZooTycoonUltimateAnimalCollection,2018,,85,7.7,8,598.0,57,19.99,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5847,KingdomUnderFire2,2019,,85,7.7,8,1399.0,73,9.99,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5848,Thief,2014,,85,7.7,8,11296.0,72,19.99,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5849,STARWARSTheForceUnleashedUltimateSithEdition,2009,,85,7.7,8,3805.0,69,19.99,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.to_csv("new_data")